---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the Jupyter Notebook FAQ course resource._

---

# Assignment 4 - Predicting and understanding viewer engagement with educational videos 

With the accelerating popularity of online educational experiences, the role of online lectures and other educational video continues to increase in scope and importance. Open access educational repositories such as <a href="http://videolectures.net/">videolectures.net</a>, as well as Massive Open Online Courses (MOOCs) on platforms like Coursera, have made access to many thousands of lectures and tutorials an accessible option for millions of people around the world. Yet this impressive volume of content has also led to a challenge in how to find, filter, and match these videos with learners. This assignment gives you an example of how machine learning can be used to address part of that challenge.

## About the prediction problem

One critical property of a video is engagement: how interesting or "engaging" it is for viewers, so that they decide to keep watching. Engagement is critical for learning, whether the instruction is coming from a video or any other source. There are many ways to define engagement with video, but one common approach is to estimate it by measuring how much of the video a user watches. If the video is not interesting and does not engage a viewer, they will typically abandon it quickly, e.g. only watch 5 or 10% of the total. 

A first step towards providing the best-matching educational content is to understand which features of educational material make it engaging for learners in general. This is where predictive modeling can be applied, via supervised machine learning. For this assignment, your task is to predict how engaging an educational video is likely to be for viewers, based on a set of features extracted from the video's transcript, audio track, hosting site, and other sources.

We chose this prediction problem for several reasons:

* It combines a variety of features derived from a rich set of resources connected to the original data;
* The manageable dataset size means the dataset and supervised models for it can be easily explored on a wide variety of computing platforms;
* Predicting popularity or engagement for a media item, especially combined with understanding which features contribute to its success with viewers, is a fun problem but also a practical representative application of machine learning in a number of business and educational sectors.


## About the dataset

We extracted training and test datasets of educational video features from the VLE Dataset put together by researcher Sahan Bulathwela at University College London. 

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single educational video, and includes information about diverse properties of the video content as described further below. The target variable is `engagement` which was defined as True if the median percentage of the video watched across all viewers was at least 30%, and False otherwise.

Note: Any extra variables that may be included in the training set are simply for your interest if you want an additional source of data for visualization, or to enable unsupervised and semi-supervised approaches. However, they are not included in the test set and thus cannot be used for prediction. **Only the data already included in your Coursera directory can be used for training the model for this assignment.**

For this final assignment, you will bring together what you've learned across all four weeks of this course, by exploring different prediction models for this new dataset. In addition, we encourage you to apply what you've learned about model selection to do hyperparameter tuning using training/validation splits of the training data, to optimize the model and further increase its performance. In addition to a basic evaluation of model accuracy, we've also provided a utility function to visualize which features are most and least contributing to the overall model performance.

**File descriptions** 
    assets/train.csv - the training set (Use only this data for training your model!)
    assets/test.csv - the test set
<br>

**Data fields**

train.csv & test.csv:

    title_word_count - the number of words in the title of the video.
    
    document_entropy - a score indicating how varied the topics are covered in the video, based on the transcript. Videos with smaller entropy scores will tend to be more cohesive and more focused on a single topic.
    
    freshness - The number of days elapsed between 01/01/1970 and the lecture published date. Videos that are more recent will have higher freshness values.
    
    easiness - A text difficulty measure applied to the transcript. A lower score indicates more complex language used by the presenter.
    
    fraction_stopword_presence - A stopword is a very common word like 'the' or 'and'. This feature computes the fraction of all words that are stopwords in the video lecture transcript.
    
    speaker_speed - The average speaking rate in words per minute of the presenter in the video.
    
    silent_period_rate - The fraction of time in the lecture video that is silence (no speaking).
    
train.csv only:
    
    engagement - Target label for training. True if learners watched a substantial portion of the video (see description), or False otherwise.
    

## Evaluation

Your predictions will be given as the probability that the corresponding video will be engaging to learners.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model with an AUC (area under ROC curve) of at least 0.8 passes this assignment, and over 0.85 will receive full points.
___

For this assignment, create a function that trains a model to predict significant learner engagement with a video using `asset/train.csv`. Using this model, return a Pandas Series object of length 2309 with the data being the probability that each corresponding video from `readonly/test.csv` will be engaging (according to a model learned from the 'engagement' label in the training set), and the video index being in the `id` field.

Example:

    id
       9240    0.401958
       9241    0.105928
       9242    0.018572
                 ...
       9243    0.208567
       9244    0.818759
       9245    0.018528
             ...
       Name: engagement, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out and check your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. 

* Try to avoid global variables. If you have other functions besides engagement_model, you should move those functions inside the scope of engagement_model.

* Be sure to first check the pinned threads in Week 4's discussion forum if you run into a problem you can't figure out.

### Extensions

* If this prediction task motivates you to explore further, you can find more details here on the original VLE dataset and others related to video engagement: https://github.com/sahanbull/VLE-Dataset



In [212]:
import warnings

warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

np.random.seed(
    0
)  # Do not change this value: required to be compatible with solutions generated by the autograder.

## Get data

Get data from training file and split into test / training data.

In [213]:
df_train = pd.read_csv(r"assets/train.csv", header=0)
X_df_train = df_train.iloc[:, :-1]
y_df_train = df_train.iloc[:, -1]
X_train, X_val, y_train, y_val = train_test_split(
    X_df_train, y_df_train, test_size=0.25, random_state=0
)

df_test = pd.read_csv(r"assets/test.csv", header=0)
X_test = df_test.iloc[:, :-1]
y_test = df_test.iloc[:, -1]

In [214]:
## Define the models and parameters

models = {
    "LogisticRegression": LogisticRegression(),
    "RandomForest": RandomForestClassifier(),
    "SVC": SVC(),
    "KNN": KNeighborsClassifier(),
    "adaBoost": AdaBoostClassifier(),
    "neural_network": MLPClassifier(),
}

# Define the parameters
params = {
    "LogisticRegression": {"clf__penalty": ["l1", "l2"], "clf__C": [0.01, 0.1, 1, 10]},
    "RandomForest": {
        "clf__n_estimators": [50, 100, 200],
        "clf__max_depth": [None, 10, 20, 30],
    },
    "SVC": {
        "clf__C": [0.01, 0.1, 1, 10, 20],
        "clf__kernel": ["linear", "rbf"],
        "clf__probability": [True],
    },
    "KNN": {"clf__n_neighbors": [1, 3, 5, 7, 9]},
    "adaBoost": {
        "clf__n_estimators": [50, 100, 200],
        "clf__learning_rate": [0.001, 0.01, 0.1, 1],
    },
    "neural_network": {
        "clf__alpha": [0.01, 0.1, 1],
        "clf__hidden_layer_sizes": [(1000,), (1000, 1000), (100,)],
    },
}

## Create Pipelines

In [215]:
pipelines = {}

for model_name in models:
    pipelines[model_name] = Pipeline(
        [("scaler", StandardScaler()), ("clf", models[model_name])]
    )

## Use GridSearchCV for model selection and hyperparameter tuning

In [216]:
grid_searches = {}

for model_name in pipelines:
    grid_search = GridSearchCV(
        pipelines[model_name],
        params[model_name],
        cv=5,
        n_jobs=-1,
        verbose=1,
        scoring="roc_auc",
    )
    grid_search.fit(X_train, y_train)
    grid_searches[model_name] = grid_search

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits


## Evaluate the models

In [218]:
best_model_name = None
best_auc = 0
for model_name, grid_search in grid_searches.items():
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    y_pred_proba = grid_search.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_pred_proba)
    print(f"AUC for {model_name}: {auc}")

    if auc > best_auc:
        best_auc = auc
        best_model_name = model_name

print(f"\nBest model is {best_model_name} with AUC: {best_auc}")

Best parameters for LogisticRegression: {'clf__C': 0.1, 'clf__penalty': 'l2'}
AUC for LogisticRegression: 0.8313303855211369
Best parameters for RandomForest: {'clf__max_depth': None, 'clf__n_estimators': 200}
AUC for RandomForest: 0.8715416728422508
Best parameters for SVC: {'clf__C': 1, 'clf__kernel': 'rbf', 'clf__probability': True}
AUC for SVC: 0.8136949755446865
Best parameters for KNN: {'clf__n_neighbors': 9}
AUC for KNN: 0.7964990613112
Best parameters for adaBoost: {'clf__learning_rate': 0.1, 'clf__n_estimators': 200}
AUC for adaBoost: 0.8508153829685621
Best parameters for neural_network: {'clf__alpha': 0.1, 'clf__hidden_layer_sizes': (100,)}
AUC for neural_network: 0.8823695304909178

Best model is neural_network with AUC: 0.8823695304909178


In [219]:
def engagement_model():
    rec = None

    # YOUR CODE HERE
    df_train = pd.read_csv("assets/train.csv", header=0)
    df_test = pd.read_csv("assets/test.csv", header=0)
    cols = [
        "title_word_count",
        "document_entropy",
        "freshness",
        "easiness",
        "fraction_stopword_presence",
        "normalization_rate",
        "speaker_speed",
        "silent_period_rate",
    ]
    X_train = df_train[cols]
    y_train = df_train.iloc[:, -1]
    X_test = df_test[cols]
    y_test = df_test.iloc[:, -1]

    clf = MLPClassifier(alpha=0.1, hidden_layer_sizes=(100,)).fit(X_train, y_train)
    rec = pd.Series(clf.predict_proba(X_test)[:, 1])
    rec.index = df_test["id"]

    return rec

In [220]:
engagement_model()

id
9240     0.390770
9241     0.884770
9242     0.998684
9243     0.997081
9244     0.026070
           ...   
11544    0.494108
11545    0.292727
11546    0.148003
11547    0.997043
11548    0.972614
Length: 2309, dtype: float64

In [221]:
stu_ans = engagement_model()
assert isinstance(stu_ans, pd.Series), "Your function should return a pd.Series. "
assert len(stu_ans) == 2309, "Your series is of incorrect length: expected 2309 "
assert np.issubdtype(
    stu_ans.index.dtype, np.integer
), "Your answer pd.Series should have an index of integer type representing video id."